In [5]:
import pandas as pd

In [6]:
import os
import random
import numpy as np
import torch

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="1"
device = 'cuda' if torch.cuda.is_available() else 'cpu'


torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

In [7]:
df = pd.read_csv('../../data/100_sentiment_analysis_sentences.csv')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    100 non-null    object
 1   label   100 non-null    object
dtypes: object(2)
memory usage: 1.7+ KB


In [9]:
# replacing values
df['label'].replace(['POSITIVE', 'NEGATIVE', 'NEUTRAL'],
                        [2, 0,1], inplace=True)

In [10]:
from sentence_transformers import SentenceTransformer

In [11]:
encoder_model = SentenceTransformer('all-MiniLM-L6-v2')

#Our sentences we like to encode
sentences = list(df['text'])

#Sentences are encoded by calling model.encode()
sentence_embeddings = encoder_model.encode(sentences)

In [12]:
sentence_embeddings.shape

(100, 384)

In [13]:
type(sentence_embeddings)

numpy.ndarray

In [14]:
#Dataset preparation
from torch.utils.data import Dataset, TensorDataset,DataLoader
from sklearn.model_selection import train_test_split


X = torch.tensor(sentence_embeddings)
y = torch.tensor(np.array(df.label.values))

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)

train_data = TensorDataset(X_train, y_train)

train_loader = DataLoader(train_data,batch_size=16, shuffle=True)

In [15]:
#sentence_embeddings[0]
type(torch.tensor(sentence_embeddings)[0])

torch.Tensor

In [16]:
y_train.shape

torch.Size([80])

In [17]:
X_train.shape

torch.Size([80, 384])

In [20]:
class Network(torch.nn.Module):
    def __init__(self,vector_size,hidden_units,num_classes): 
      super().__init__()
      #First fully connected layer
      self.fc1 = torch.nn.Linear(vector_size,hidden_units)
      #Second fully connected layer
      self.fc2 = torch.nn.Linear(hidden_units,num_classes)
      #Final output of softmax function      
      self.output = torch.nn.Softmax()
    def forward(self,x):
      fc1 = self.fc1(x)
      fc2 = self.fc2(fc1)
      output = self.output(fc2)
      #return output[:, -1]
      return output

In [21]:
NUM_EPOCHS = 5
VECTOR_SIZE = X.shape[1]
HIDDEN_UNITS = 4
OUT_CLASSES = 3
LEARNING_RATE = 0.001
#Initialize model
custom_model = Network(VECTOR_SIZE,HIDDEN_UNITS,OUT_CLASSES)
print(custom_model)
#Initialize optimizer
optimizer =torch.optim.SGD(custom_model.parameters(), lr=LEARNING_RATE)
#Initialize loss function
loss_fun = torch.nn.CrossEntropyLoss()
for i in range(NUM_EPOCHS):
   for x_batch,y_batch in train_loader:
       custom_model.train()
       y_pred = custom_model(x_batch)
       #print(y_pred)
       loss = loss_fun(y_pred,y_batch)
       loss.backward()
       optimizer.step()
       optimizer.zero_grad()
   print('After {} epoch training loss is {}'.format(i,loss.item()))

Network(
  (fc1): Linear(in_features=384, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=3, bias=True)
  (output): Softmax(dim=None)
)
After 0 epoch training loss is 1.1138381958007812
After 1 epoch training loss is 1.092646837234497
After 2 epoch training loss is 1.0765480995178223
After 3 epoch training loss is 1.0851796865463257
After 4 epoch training loss is 1.1110912561416626


C:\Users\metes\AppData\Local\Temp\ipykernel_40008\2920850770.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.output(fc2)


In [22]:
#torch.save(custom_model, "../models/embedding_sentence_transformer_custom_dl")
torch.save(custom_model.state_dict(), "../models/embedding_sentence_transformer_custom_dl/model.pth")

In [23]:
#saved_model = torch.load("../models/embedding_sentence_transformer_custom_dl")
saved_model = Network(384,4,3)
saved_model.load_state_dict(torch.load("../models/embedding_sentence_transformer_custom_dl/model.pth"))


<All keys matched successfully>

In [24]:
from sentence_transformers import SentenceTransformer
sentence_embeddings = torch.tensor(SentenceTransformer('all-MiniLM-L6-v2').encode(['test_input'])) 
saved_model.eval()
outputs = saved_model(sentence_embeddings)

print(outputs)
predicted_class_id = outputs.argmax().item()
print(predicted_class_id)

tensor([[0.3432, 0.3779, 0.2789]], grad_fn=<SoftmaxBackward0>)
1


C:\Users\metes\AppData\Local\Temp\ipykernel_40008\2920850770.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.output(fc2)
